In [1]:
from odc.stac import configure_s3_access, load
from pystac_client import Client as PystacClient
import numpy as np
import odc.geo.xr


In [2]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43959,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.61 GiB
Comm: tcp://127.0.0.1:46831,Total threads: 1
Dashboard: http://127.0.0.1:39391/status,Memory: 3.90 GiB
Nanny: tcp://127.0.0.1:37257,


In [3]:
catalog = "https://explorer.dea.ga.gov.au/stac"

stac_client = PystacClient.open(catalog)

configure_s3_access(
    cloud_defaults=True,
    aws_unsigned=True,
)

In [11]:
date_query = ['2023-07','2024-06']

collections_query = ['ga_s2am_ard_3', 'ga_s2bm_ard_3']

filter_query = "eo:cloud_cover < 10"



import geopandas as gpd
gdf = gpd.read_file('lower_balonne.shp')

geom= gdf.__geo_interface__

geom

In [12]:
items = stac_client.search(
    collections = collections_query,
    bbox= [147.0282090,-29.6753190,148.745019431,-30.0063751],
    datetime = date_query,
    filter= filter_query
).item_collection()

print(f"Found {len(items)} items")

#bbox south [145.910476061,-29.153361653,148.745019431,-30.128112653]
#bbox north [146.963236949,-27.723499515,148.806899103,-29.169637680]
#bbox narran[147.0282090,-29.6753190,148.745019431,-30.0063751]

Found 400 items


In [13]:
ds_filtered = load(
    items,
    bands=['nbart_red', 'nbart_green', 'nbart_nir_1',
                     'nbart_swir_2', 'nbart_swir_3'],
    crs="utm",
    chunks={"time": 109, "x": 400, "y": 400},
    resolution=30,
    groupby="solar_day",
    bbox= [145.910476061,-29.153361653,148.745019431,-30.128112653],
)

ds_filtered

<xarray.Dataset> Size: 73GB
Dimensions:       (y: 3627, x: 9191, time: 109)
Coordinates:
  * y             (y) float64 29kB 6.775e+06 6.774e+06 ... 6.666e+06 6.666e+06
  * x             (x) float64 74kB 3.94e+05 3.941e+05 ... 6.697e+05 6.697e+05
    spatial_ref   int32 4B 32755
  * time          (time) datetime64[ns] 872B 2023-07-09T00:25:06.147214 ... 2...
Data variables:
    nbart_red     (time, y, x) float32 15GB dask.array<chunksize=(109, 400, 400), meta=np.ndarray>
    nbart_green   (time, y, x) float32 15GB dask.array<chunksize=(109, 400, 400), meta=np.ndarray>
    nbart_nir_1   (time, y, x) float32 15GB dask.array<chunksize=(109, 400, 400), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 15GB dask.array<chunksize=(109, 400, 400), meta=np.ndarray>
    nbart_swir_3  (time, y, x) float32 15GB dask.array<chunksize=(109, 400, 400), meta=np.ndarray>

In [15]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
dsfwi =1.7204 + 171*(ds_filtered.nbart_green/10000) + 3*(ds_filtered.nbart_red/10000) - 70*(ds_filtered.nbart_nir_1/10000) - 45*(ds_filtered.nbart_swir_2/10000)-71*(ds_filtered.nbart_swir_3/10000)

dsfwi

<xarray.DataArray (time: 109, y: 3627, x: 9191)> Size: 15GB
dask.array<sub, shape=(109, 3627, 9191), dtype=float32, chunksize=(109, 400, 400), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 29kB 6.775e+06 6.774e+06 ... 6.666e+06 6.666e+06
  * x            (x) float64 74kB 3.94e+05 3.941e+05 ... 6.697e+05 6.697e+05
    spatial_ref  int32 4B 32755
  * time         (time) datetime64[ns] 872B 2023-07-09T00:25:06.147214 ... 20...

In [ ]:
# Resample using medians
print("Calculating FWI seasonal maxima...")
annualmaxfwi = dsfwi.quantile(.95,'time').compute()

annualmaxfwi

Calculating FWI seasonal maxima...


/home/vscode/.local/lib/python3.11/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 12.11 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [ ]:
annualmaxfwi.odc.write_cog(
    "fwimax_2324_south.tif",
    overwrite = True
)

PosixPath('autumn9095pFWI_north.tif')